In [13]:
import os
import json
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, Embedding

In [14]:
char_to_index_file = "./Data/char_to_index.json"
model_weights_dir = './Data/Model_Weights/'

## Functions

In [15]:
def make_model(unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (1, 1))) 
  
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, stateful = True)) #just have to get 1 output no need for return_sequences=True
    model.add(Dropout(0.2))
    
    model.add((Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    return model

In [49]:
def generate_sequence(epoch_num, initial_index, seq_length):
    with open(char_to_index_file) as f:
        char_to_index = json.load(f)
        
    index_to_char = {i:ch for ch, i in char_to_index.items()}
    unique_chars = len(index_to_char)
    
    model = make_model(unique_chars)
    model.load_weights(model_weights_dir + "Weights_{}.h5".format(epoch_num))
     
    sequence_index = [initial_index]
    
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(unique_chars), size = 1, p = predicted_probs)
        
        sequence_index.append(sample[0])
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    
    for i in range(len(seq)):
        if seq[i] == "\n":
            break
    seq1 = seq[i+1:]
    # Ignores the first (meaningless) string before \n of the sequence, generated by a character input. 
    # Only consider the tune composed by model from next line onwards. 
    
    for i in range(len(seq1)):
        if seq1[i] == "\n" and seq1[i+1] == "\n":
            break
    seq2 = seq1[:i+1]
    # Ignores all characters generated by model after consecutive new line characters since data has three newline characters after every tune. 
    # Only composes one tune at a time
    
    return seq2

## Compose Music

In [59]:
epoch = int(input("Enter epoch weight no. to load (10,20,...,90) - the higher, the better: "))
char = int(input("\nEnter an integer (0-86) as initial input: "))
length = int(input("\nEnter the desired length of music sequence (300~600): "))

music = generate_sequence(epoch, char, length)

print("\nMUSIC SEQUENCE GENERATED: \n")
# \nX:1\nT:Music Output

print(music)

Enter epoch weight no. to load (10,20,...,90) - the higher, the better: 90

Enter an integer (0-86) as initial input: 67

Enter the desired length of music sequence (300~600): 600

MUSIC SEQUENCE GENERATED: 

% Nottingham Music Database
S:John Kerkpatrick, via EF
M:6/8
K:A
P:A
E|"A"A2E A2c|"A"e2e ece|"D"fga "A"ecA|"G"A3 "A"A2A|
"D"a2f "A"e3|"D"fed "E7"c2B|"A"A3 A2:|[2"A"A3 A2:|[2 "A" e3 A3|
P:B
P:B
|:"D"AFA fed|"A"cAA eAc|"A"edc "E"dBG|"A"A3 -A2:|



### Convert to MID files via this link,  download and play with GarageBand (macOS)
https://www.mandolintab.net/abcconverter.php